In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re

search_n = input("원하는 분야를 입력해주세요!>")
#saveFileNm = input("저장할 파일 이름을 정해주세요>")

#search_n = "데이터분석"
url ="http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+search_n+"&loc_mcd=101000&company_cd=0,1,2,3,4,5,6,7,9&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage=2&recruitSort=relation&recruitPageCount=50&inner_com_type=&quick_apply="
urlsrc = requests.get(url)
soup = bs(urlsrc.text, "html.parser")

#불러올 데이터 총 개수 찾기!
cnt_result = soup.find("span", class_="cnt_result").text
cnt_result = re.sub(r"총 |건","",cnt_result)
cnt_result = cnt_result.split(",")
totCnt = ""
for i in range(len(cnt_result)):
    totCnt += cnt_result[i]
totCnt = int(totCnt)
#불러올 데이터 수!
totCnt

#불러올 페이지수
totPage = totCnt//50 +1
totPage


#DF선언
saramIn_df = pd.DataFrame()
saramIn_df_temp = pd.DataFrame()
saramIn_df_temp = saramIn_df_temp.append(
                {"coID":"",
                 "coNm":"",
                 "r_date":"",
                 "loc_city":"",
                 "loc_detail":"",
                 "cod_a":"",
                 "cod_b":"",
                 "cod_c":"",
                 "content":"",
                 "link":"",
                 "coIndx":"",
                 #"coIndx":"",
                }, ignore_index=True
)
for page in range(1,totPage+1):
    URL = "http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+search_n+"&loc_mcd=101000&company_cd=0,1,2,3,4,5,6,7,9&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage="+str(page)+"&recruitSort=relation&recruitPageCount=50&inner_com_type=&quick_apply="
    print(URL)
    urlsrc = requests.get(URL)
    
    soup = bs(urlsrc.text, "html.parser")
    comp_list = soup.find_all("div",class_="item_recruit")
    
    for i in range(len(comp_list)):
        content = comp_list[i].find("h2").find("a").get("title")
        content = re.sub(r" ","",content)
        link = comp_list[i].find("h2").find("a").get("href")
        link = "http://www.saramin.co.kr/" + link
        r_date = comp_list[i].find("span", class_="date").text
        r_date = re.sub(r"~ ","",r_date)
        loc_city = comp_list[i].find("div", class_="job_condition").find_all("span")[0].find_all("a")[0].text
        #loc_detail = comp_list[i].find("div", class_="job_condition").find_all("span")[0].find_all("a")[1].text
        loc_detail=""
        if len(comp_list[i].find("div", class_="job_condition").find_all("span")) >= 4:
            
            cod_a =  comp_list[i].find("div", class_="job_condition").find_all("span")[1].text
            cod_b = comp_list[i].find("div", class_="job_condition").find_all("span")[2].text
            if cod_b != "학력무관":
                cod_b = re.sub(r"↑","",cod_b)
            cod_c = comp_list[i].find("div", class_="job_condition").find_all("span")[3].text
            cod_c = re.sub(r"·","",cod_c)
        elif len(comp_list[i].find("div", class_="job_condition").find_all("span")) < 4:
            cod_a =  comp_list[i].find("div", class_="job_condition").find_all("span")[1].text
            cod_b = comp_list[i].find("div", class_="job_condition").find_all("span")[2].text
            if cod_b != "학력무관":
                cod_b = re.sub(r"↑","",cod_b)
            cod_c = ""
        coID = comp_list[i].find("div", class_="area_corp").find("div", class_="area_btn").get("value")
        companyNm = comp_list[i].find("div", class_="area_corp").strong.a.get("title")
        coIndx = comp_list[i].find("h2",class_="job_tit").find_all("a")[0].get("href")
        coIndx = re.findall(r"[0-9]{8}",coIndx)[0]
        
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coID")] = coID
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coNm")] = companyNm 
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("r_date")] = r_date
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("loc_city")] = loc_city
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("loc_detail")] = loc_detail
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cod_a")] = cod_a
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cod_b")] = cod_b
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cod_c")] = cod_c
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("content")] = content
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("link")] = link
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coIndx")] = coIndx
        
        saramIn_df = saramIn_df.append(saramIn_df_temp, ignore_index= True)
        saramIn_df_temp.drop(0)

        
        

saramIn_df.to_excel("C:\\Users\\roufs\\Desktop\\recruit\\saramIn07081.xlsx")

saramIn_df


        
        
    
    
    
    
    


In [ ]:
saramIn_df.rename({"cod_a":"경력조건"},axis = 1,inplace=True)
saramIn_df.rename({"cod_b":"학력조건","content":"제목","coNm":"회사명","loc_city":"지역","r_date":"모집기간"},axis=1,inplace=True)
saramIn_df[saramIn_df['학력조건'] == '대졸']

In [ ]:
coIndx=comp_list[0].find("h2",class_="job_tit").find_all("a")[0].get("href")
coIndx = re.findall(r"[0-9]{8}",coIndx)[0]
coIndx

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
from selenium import webdriver

saramIn_nu_df = pd.DataFrame()
saramIn_nu_df_temp = pd.DataFrame()
saramIn_nu_df_temp = saramIn_nu_df_temp.append(
                {"coIndx":"",
                 "scrap_cnt":"",
                 "view_cnt":"",
                 #"r_date":"",
                 #"s_date":"",
                 "apply_cnt":"",
                 "apply_male":"",
                 "apply_female":"",
                 "apply_20s":"",
                 "apply_30s":"",
                 "apply_40s":"",
                 #"coIndx":"",
                }, ignore_index=True
)

driver = webdriver.Chrome("C:/Users/roufs/Desktop/myPython/chromedriver.exe")

for i in saramIn_df["coIndx"]:
    url = "http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx="+"38356726"+"&recommend_ids=eJxdkMsRQyEMA6vJ3diWP%2BcUQv9dhISHzeS4swJpLKHMEjRh4%2BVvCSXxwETqxgGBLsQPl1PlCjfOHRcQkup14%2BOZk1inpW2fgby%2BK5zVzprdng7h6Wp%2FeOLMrj22sTwFxeUPVl2CqH3h8QQFqr%2Fx8cPFR5Zf09eFphHvuba8XKhu1kjOzoU8jPIK%2BxD%2FHkI%2FuiBfJA%3D%3D&view_type=search&searchword=데이터분석searchType=default_mysearch&gz=1&t_ref_content=generic&"
    driver.get(url)
    if driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/div/button[2]/span""") != None:
        scrap_cnt = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/div/button[2]/span""")[0].text
    else:
        pass
    view_cnt_samp = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[2]/ul/li[1]/strong""")[0].text
    view_cnt_samp = view_cnt_samp.split(",")
    view_cnt =""
    for i in range(len(view_cnt_samp)):
        view_cnt += view_cnt_samp[i]
    #if driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[4]/div[2]/div/dl/dd[2]""") !=None:
        #r_date= driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[4]/div[2]/div/dl/dd[2]""")[0].text
        #//*[@id="content"]/div[2]/div[1]/div[1]/div[6]/div[2]/div/dl/dd[2]
    #else:
        #pass
    #s_date= driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[4]/div[2]/div/dl/dd[1]""")[0].text
    apply_cnt = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[1]/dl/dd/span""")[0].text
    apply_male = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[2]/div/div/dl[1]/dd[2]""")[0].text
    apply_male = re.sub(r" 명","",apply_male)
    apply_female = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[2]/div/div/dl[2]/dd[2]""")[0].text
    apply_female = re.sub(r" 명","",apply_female)
    apply_20s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[1]/div/div/span""")[0].text
    apply_30s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[2]/div/div/span""")[0].text
    apply_40s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[3]/div/div/span""")[0].text
    
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("coIndx")] = i
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("scrap_cnt")] = scrap_cnt
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("view_cnt")] = view_cnt
    #saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("r_date")] = r_date
    #saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("s_date")] = s_date
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_cnt")] = apply_cnt
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_male")] = apply_male
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_female")] = apply_female
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_20s")] = apply_20s
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_30s")] = apply_30s
    saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_40s")] = apply_40s
    
    saramIn_nu_df = saramIn_nu_df.append(saramIn_nu_df_temp,ignore_index=True)
    print(str(i),"완료")
    saramIn_nu_df_temp.drop(0)
    
saramIn_nu_df
    
    
    

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
from selenium import webdriver

saramIn_nu_df = pd.DataFrame()
saramIn_nu_df_temp = pd.DataFrame()
saramIn_nu_df_temp = saramIn_nu_df_temp.append(
                {"coIndx":"",
                 "scrap_cnt":"",
                 "view_cnt":"",
                 "r_date":"",
                 "s_date":"",
                 "apply_cnt":"",
                 "apply_male":"",
                 "apply_female":"",
                 "apply_20s":"",
                 "apply_30s":"",
                 "apply_40s":"",
                 #"coIndx":"",
                }, ignore_index=True
)

driver = webdriver.Chrome("C:/Users/roufs/Desktop/myPython/chromedriver.exe")

url = "http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx="+"38356726"+"&recommend_ids=eJxdkMsRQyEMA6vJ3diWP%2BcUQv9dhISHzeS4swJpLKHMEjRh4%2BVvCSXxwETqxgGBLsQPl1PlCjfOHRcQkup14%2BOZk1inpW2fgby%2BK5zVzprdng7h6Wp%2FeOLMrj22sTwFxeUPVl2CqH3h8QQFqr%2Fx8cPFR5Zf09eFphHvuba8XKhu1kjOzoU8jPIK%2BxD%2FHkI%2FuiBfJA%3D%3D&view_type=search&searchword=데이터분석searchType=default_mysearch&gz=1&t_ref_content=generic&"
driver.get(url)
if driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/div/button[2]/span""") != None:
    scrap_cnt = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/div/button[2]/span""")[0].text
else:
    pass
view_cnt_samp = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[2]/div[2]/ul/li[1]/strong""")[0].text
view_cnt_samp = view_cnt_samp.split(",")
view_cnt =""
for i in range(len(view_cnt_samp)):
    view_cnt += view_cnt_samp[i]

r_date= driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[4]/div[2]/div/dl/dd[2]""")[0].text
s_date= driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[4]/div[2]/div/dl/dd[1]""")[0].text
apply_cnt = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[1]/dl/dd/span""")[0].text
apply_male = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[2]/div/div/dl[1]/dd[2]""")[0].text
apply_male = re.sub(r" 명","",apply_male)
apply_female = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[2]/div/div/dl[2]/dd[2]""")[0].text
apply_female = re.sub(r" 명","",apply_female)
apply_20s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[1]/div/div/span""")[0].text
apply_30s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[2]/div/div/span""")[0].text
apply_40s = driver.find_elements_by_xpath("""//*[@id="content"]/div[2]/div[1]/div[1]/div[5]/div[2]/div/div/div[3]/div/div[3]/div/div/span""")[0].text
    
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("coIndx")] = i
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("scrap_cnt")] = scrap_cnt
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("view_cnt")] = view_cnt
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("r_date")] = r_date
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("s_date")] = s_date
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_cnt")] = apply_cnt
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_male")] = apply_male
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_female")] = apply_female
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_20s")] = apply_20s
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_30s")] = apply_30s
saramIn_nu_df_temp.iat[0,saramIn_nu_df_temp.columns.get_loc("apply_40s")] = apply_40s

saramIn_nu_df_temp
    

s_df= saramIn_df.set_index("coID")

print(len(s_df.index.unique()))
print(s_df.shape[0])

In [ ]:
# "키워드 값만 찾아오기!"

In [ ]:
keyword_01 = input("첫번째 키워드를 입력하세요")
keyword_02 = input("두번째 키워드를 입력하세요")
keyword_03 = input("세번째 키워드를 입력하세요")

len(saramIn_df['제목'])
data_analyist_list =[]
for i in range(len(saramIn_df['제목'])):
    if keyword_01 in saramIn_df['제목'][i] or "keyword_02" in saramIn_df['제목'][i] or keyword_03 in saramIn_df['제목'][i]:
        data_analyist_list.append(i)
    else:
        continue

saramIn_key_df =saramIn_df.loc[data_analyist_list,:]

saramIn_key_df.to_excel("C:\\Users\\roufs\\Desktop\\recruit\\saramIn_keyword.xlsx")
        

In [ ]:
not_d = []

for i in range(len(saramIn_df['제목'])):
    if "개발자" in saramIn_df['제목'][i]:
        continue
    else:
        not_d.append(i)

saramIn_df.loc[not_d,:]
        